In [1]:
ra_s = 166.113808  # ra in degree
dec_s = 38.208833 # dec in degree
image_size = 3. # in arcmin (integrate flux of all sources within this radius)
radius_photometry = 1. # in arcsec
dr=9 # data release
image_band='g' #
pixsize=1. # arcsec per pixel

In [2]:
import numpy as np
from numpy import pi
import os

# Conventional astronomical tools, also to be traced by Renku plugin, there is domain-specific ontology built in
from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import Angle
import astropy.units as u
from astropy.constants import hbar

nano_maggies_to_Jy = 3.631e-6 # constant of conversion
if not(os.path.isdir('figs')):
    os.makedirs('figs')
if not(os.path.isdir('data')):
    os.makedirs('data')

from astropy.table import Table
import hashlib
from astroquery.desi import DESILegacySurvey

In [3]:
radius_photometry = Angle(radius_photometry * u.arcsec)
image_size = Angle(image_size * u.arcmin)
pixsize = Angle(pixsize * u.arcsec)
npix=int(2*image_size/pixsize)
source = SkyCoord(ra_s,dec_s,unit='degree')

In [4]:
query=DESILegacySurvey.get_images(coordinates='icrs', radius=image_size, pixels=npix, survey='dr%d'%dr, position=source)
hdul = query[0]
hdul[0].header

image_url:  https://www.legacysurvey.org/viewer/fits-cutout?ra=166.113808&dec=38.208833&size=360&layer=ls-dr9&pixscale=1.0&bands=g


SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  360 / length of data axis 1                          
NAXIS2  =                  360 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BANDS   = 'g       '                                                            
BAND0   = 'g       '                                                            
CTYPE1  = 'RA---TAN'           / TANgent plane                                  
CTYPE2  = 'DEC--TAN'           / TANgent plane                                  
CRVAL1  =           166.1138

In [5]:
paramstring='ra='+str(ra_s)+'&dec='+str(dec_s)+'&size='+str(npix)+'&layer=ls-dr'+str(dr)+'&pixscale='+str(pixsize)+'&bands='+image_band
suffix = hashlib.md5(paramstring.encode()).hexdigest()
filename='data/image_legacysurvey_%s.fits'%( suffix )

if os.path.exists(filename):
        os.remove(filename)
hdul.writeto(filename)

In [6]:
hdu = hdul[0]
image=hdu.data
wcs=WCS(hdu.header)
w = WCS(hdu.header)
sky = w.pixel_to_world(0, 0)
ra_max_image=sky.ra.degree
dec_min_image=sky.dec.degree
sky = w.pixel_to_world(npix-1, npix-1)
ra_min_image=sky.ra.degree
dec_max_image=sky.dec.degree

In [7]:
query = DESILegacySurvey.query_region(coordinates=source, radius=image_size, data_release=dr)

tap_result = query
ra=tap_result['ra']
dec=tap_result['dec']
t=tap_result['type']
nsources=len(tap_result)
coord = SkyCoord(ra=ra, dec=dec, unit='deg',frame='icrs')
sep=source.separation(coord)
indexes=[]
for i in range(nsources):
    if(t[i]=='DUP'):
        sep[i]=100. * u.deg
        # TODO: this is just to ignore it from table, rather just delete from it
    if(sep[i]< radius_photometry): # FIXME: parameter here (or maybe not)
        indexes.append(i)
minsep=np.min(sep)
index=np.argmin(sep)
print('found LegacySurvey source number',indexes,'at separation',sep[indexes],'arcsec')
print(minsep,index)
ra_min=min(ra)
dec_min=min(dec)
ra_max=max(ra)
dec_max=max(dec)
print(min(ra),max(ra),min(dec),max(dec))
print(ra_min_image,ra_max_image,dec_min_image,dec_max_image)

tap_result.write('data/catalog_legacysurvey_%s.ecsv'%suffix, overwrite=True)

found LegacySurvey source number [302] at separation [0d00m00.08183811s] arcsec
0d00m00.08183811s 302
166.05017663959927 166.17706337477026 38.15890988931406 38.258697231549064
166.050308748095 166.1772203138659 38.158954853903886 38.258676989793564


In [8]:
label=['g','r','z','w1','w2','w3','w4']
wavelength=np.array([4770,6231,9134,3.368e4,4.618e4,12.082e4,22.194e4])
wavelength=wavelength*1e-8 # in cm
frequency=3.e10/wavelength # in Hz
energy=2*pi*hbar.to('eV s')/u.eV/u.s*frequency # in eV
factor=1e-23*frequency # conversion of Jy to erg/cm2s
flux=np.zeros(len(label))
err=np.zeros(len(label))
for i in range(len(label)):
    for j in range(len(indexes)):
        if (tap_result['flux_ivar_'+label[i]][indexes[j]]>0):
            flux[i]+=tap_result['flux_'+label[i]][indexes[j]]
            err[i]+=1 / tap_result['flux_ivar_'+label[i]][indexes[j]]

    flux[i]=flux[i]/tap_result['mw_transmission_'+label[i]][index]*nano_maggies_to_Jy*factor[i]
    err[i]=np.sqrt(err[i])/tap_result['mw_transmission_'+label[i]][index]*nano_maggies_to_Jy*factor[i]
    print(energy[i],flux[i],err[i])

data = Table()
data['E'] = energy*u.eV
data['nuFnu'] = flux*u.erg/u.cm**2/u.s
data['nuFnu_err'] = err*u.erg/u.cm**2/u.s
data.write('data/spectrum_legacysurvey_%s.ecsv'%suffix, overwrite=True)
data.info()

2.6010488659898483 8.121009284709404e-11 4.2498343810795666e-14
1.9911736624573224 1.2224406124990915e-10 5.911111791087553e-14
1.3583318470299512 1.2110371815123045e-10 3.09828401848189e-14
0.36837895162623446 4.2035761760794405e-11 2.1138603946206e-14
0.268666156144902 2.7025956568311665e-11 2.408139670746677e-14
0.10268997757632492 2.6660426935817343e-11 6.442100085443842e-14
0.0559025100962944 1.88067576422018e-11 2.454355722750842e-13
<Table length=7>
   name    dtype       unit    
--------- ------- -------------
        E float64            eV
    nuFnu float64 erg / (cm2 s)
nuFnu_err float64 erg / (cm2 s)


In [9]:
#here we prepare output. Images will not be forwarded in production, but now as it is
with open('data/catalog_legacysurvey_%s.ecsv'%suffix, 'r') as catfd, \
     open('data/spectrum_legacysurvey_%s.ecsv'%suffix, 'r') as specfd, \
     fits.open('data/image_legacysurvey_%s.fits'%suffix, 'readonly') as imfitsfd:
    catalog = catfd.read()
    spec = specfd.read()
    imfits_head = {k: v for k, v in imfitsfd[0].header.items()}
    imfits_data = imfitsfd[0].data

In [10]:
catalog = catalog
spec = spec
imfits_head = imfits_head
imfits_data = imfits_data